In [ ]:
# @hidden_cell

import pandas as pd
import numpy as np
import math

import geopandas
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

In [ ]:
# @hidden_cell

import geopandas as gpd

geojson = gpd.read_file('../input/cc-spring-2021/neighbourhood_data.csv')

In [ ]:
# @hidden_cell

import geoplot

count2015 = pd.read_csv('../input/cc-spring-2021/count2015.csv')
count2017 = pd.read_csv('../input/cc-spring-2021/count2017.csv')
count2019 = pd.read_csv('../input/cc-spring-2021/count2019.csv')

Plot of neighborhood boundaries and how many properties there are.

In [ ]:
import pandas as pd
import numpy as np
import math

import geopandas
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
#from bokeh.palettes import brewere
from bokeh.palettes import Category20

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

In [ ]:
neighborhood_data = pd.read_csv('../input/cc-spring-2021/neighbourhood_data.csv')

def json_data(selectedYear):
    yr = selectedYear
    
    # Pull selected year from neighborhood summary data
    df_yr = neighborhood_data[neighborhood_data['year'] == yr]
    
    # Merge the GeoDataframe object (sf) with the neighborhood summary data (neighborhood)
    merged = pd.merge(geojson, df_yr, on='neighbourhood', how='left')
    
    # Fill the null values
    values = {'year': yr, 'total_revenue': 0, 'average_revenue': 0, 'counts': 0}
    merged = merged.fillna(value=values)
    
    # Bokeh uses geojson formatting, representing geographical features, with json
    # Convert to json
    merged_json = json.loads(merged.to_json())
    
    # Convert to json preferred string-like object 
    json_data = json.dumps(merged_json)
    return json_data

# This dictionary contains the formatting for the data in the plots
format_data = [('counts', 0, 700,'0,0', 'Number of Properties'),
               ('total_revenue', 0, 40000000,'$0,0', 'Total Revenue'),
               ('average_revenue', 0, 370, '$0,0', 'Average Revenue')
              ]
 
#Create a DataFrame object from the dictionary 
format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

# Create a plotting function
def make_plot(field_name):    
  # Set the format of the colorbar
  min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
  max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
  field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]

  # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
  color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

  # Create color bar.
  format_tick = NumeralTickFormatter(format=field_format)
  color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, formatter=format_tick,
  border_line_color=None, location = (0, 0))

  # Create figure object.
  verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]

  p = figure(title = verbage + ' by Neighborhood for Rentals in SF by Year - 2015 to 2020', 
             plot_height = 650, plot_width = 850,
             toolbar_location = None)
  p.xgrid.grid_line_color = None
  p.ygrid.grid_line_color = None
  p.axis.visible = False

  # Add patch renderer to figure. 
  p.patches('xs','ys', source = geosource, fill_color = {'field' : field_name, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
  
  # Specify color bar layout.
  p.add_layout(color_bar, 'right')

  # Add the hover tool to the graph
  p.add_tools(hover)
  return p


In [ ]:
from ipywidgets import interact
import numpy as np

# This dictionary contains the formatting for the data in the plots
format_data = [('counts', 0, 700,'0,0', 'Number of Properties'),
                   ('total_revenue', 0, 32000000,'$0,0', 'Total Revenue'),
                   ('average_revenue', 0, 370, '$0,0', 'Average Revenue')
                  ]
 
# Create a DataFrame object from the dictionary 
format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

 # Define a sequential multi-hue color palette.
palette = brewer['Blues'][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Add a hover tool.
hover = HoverTool(tooltips = [ ('Neighborhood','@neighbourhood'),
                              ('Number of Properties', '@counts'),
                              ('Total Revenue', '$@total_revenue{,}'),
                              ('Average Revenue', '$@average_revenue{,}')])

# Create a plotting function
def make_plot(field_name):    
# Set the format of the colorbar
    min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
    max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
    field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]
    # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

    # Create color bar.
    format_tick = NumeralTickFormatter(format=field_format)
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, formatter=format_tick,
    border_line_color=None, location = (0, 0))
    # Create figure object.
    verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]

    p = figure(title = verbage + ' by Neighborhood for Rentals in SF by Year - 2015 to 2020', 
             plot_height = 650, plot_width = 850,
             toolbar_location = None)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False

    # Add patch renderer to figure. 
    p.patches('xs','ys', source = geosource, fill_color = {'field' : field_name, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
  
    # Specify color bar layout.
    p.add_layout(color_bar, 'right')

    # Add the hover tool to the graph
    p.add_tools(hover)
    return p

def update(type, year=2020):
    global input_field
    global geosource
    if type == 'Number of Properties':
        input_field = 'counts'
    elif type == 'Total Revenue':
        input_field = 'total_revenue'
    elif type == 'Average Revenue':
        input_field = 'average_revenue'
    if year == 2015: 
        geosource = GeoJSONDataSource(geojson = json_data(2015))
    elif year == 2016:
        geosource = GeoJSONDataSource(geojson = json_data(2016))
    elif year == 2017:
        geosource = GeoJSONDataSource(geojson = json_data(2017))
    elif year == 2018:
        geosource = GeoJSONDataSource(geojson = json_data(2018))
    elif year == 2019:
        geosource = GeoJSONDataSource(geojson = json_data(2019))
    elif year == 2020:
        geosource = GeoJSONDataSource(geojson = json_data(2020))

    # Call the plotting function
    p = make_plot(input_field)

    output_notebook()
    show(p)

In [ ]:
interact(update, type=['Number of Properties', 'Total Revenue', 'Average Revenue'], year=(2015, 2020))